In [0]:
storage_account_name = "adls1046project"
container_name = "container"
file_name = "GlobalLandTemperaturesByCountry.csv"
storage_account_access_key = "KUKAhIfCYt/M/1Co79hLgWIsSzICBxQ7TJQ3s7DwtrO1gZWe/lRA0Rh43+Qmu1+PR31WfvqtRx6w+ASthC5Ofg=="
spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)
data = spark.read.format("csv").load(f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{file_name}", inferSchema=True, header=True)

# Register the DataFrame as a SQL temporary view
data.createOrReplaceTempView("climate_data")

In [0]:
# Identify the structure and schema of the dataset
print("Schema of the dataset:")
data.printSchema()

Schema of the dataset:
root
 |-- Date: date (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
# Display basic statistics of numerical columns
print("Basic statistics of numerical columns:")
data.describe().show()

Basic statistics of numerical columns:
+-------+------------------+-----------------------------+-----------+
|summary|AverageTemperature|AverageTemperatureUncertainty|    Country|
+-------+------------------+-----------------------------+-----------+
|  count|            544811|                       545550|     577462|
|   mean| 17.19335423293594|           1.0190569003757632|       NULL|
| stddev|10.953966445121155|            1.201930386633437|       NULL|
|    min|           -37.658|         0.052000000000000005|Afghanistan|
|    max| 38.84200000000001|                       15.003|      Åland|
+-------+------------------+-----------------------------+-----------+



In [0]:
# Use Spark SQL queries to explore the data
# Example 1: Count the number of records in the dataset
record_count = spark.sql("SELECT COUNT(*) AS record_count FROM climate_data").collect()[0]["record_count"]
print("Number of records in the dataset:", record_count)

Number of records in the dataset: 577462


In [0]:
# Example 2: Show the first few rows of the dataset
print("First few rows of the dataset:")
spark.sql("SELECT * FROM climate_data LIMIT 5").show()

First few rows of the dataset:
+----------+------------------+-----------------------------+-------+
|      Date|AverageTemperature|AverageTemperatureUncertainty|Country|
+----------+------------------+-----------------------------+-------+
|1743-11-01|4.3839999999999995|                        2.294|  Åland|
|1743-12-01|              NULL|                         NULL|  Åland|
|1744-01-01|              NULL|                         NULL|  Åland|
|1744-02-01|              NULL|                         NULL|  Åland|
|1744-03-01|              NULL|                         NULL|  Åland|
+----------+------------------+-----------------------------+-------+



In [0]:
# Example 3: Display distinct values in a particular column
print("Distinct Countries:")
spark.sql("SELECT DISTINCT Country FROM climate_data").show()

Distinct Countries:
+--------------+
|       Country|
+--------------+
|      Anguilla|
|         Åland|
|   Afghanistan|
|        Africa|
|       Algeria|
|     Argentina|
|        Angola|
|  Baker Island|
|       Albania|
|       Bahamas|
|American Samoa|
|       Andorra|
|    Antarctica|
|         Aruba|
|    Azerbaijan|
|       Armenia|
|          Asia|
|     Australia|
|       Austria|
|       Bahrain|
+--------------+
only showing top 20 rows



In [0]:
# Example 4: Calculate the average value of a numerical column
print("Average value of a of temperature of the year:")
avg_temp = spark.sql("SELECT DATE_PART('YEAR', Date), AVG(AverageTemperature) AS avg_temp FROM climate_data GROUP BY DATE_PART('YEAR', Date) ORDER BY DATE_PART('YEAR', Date)")
avg_temp.show()

Average value of a of temperature of the year:
+---------------------+------------------+
|date_part(YEAR, Date)|          avg_temp|
+---------------------+------------------+
|                 1743|           5.18414|
|                 1744|         9.8378975|
|                 1745|1.3871250000000004|
|                 1746|              NULL|
|                 1747|              NULL|
|                 1748|              NULL|
|                 1749|              NULL|
|                 1750| 9.129352727272728|
|                 1751| 9.167387499999998|
|                 1752| 4.413386666666668|
|                 1753| 8.870820754716977|
|                 1754| 8.822018957345971|
|                 1755| 8.530536277602524|
|                 1756|  9.17988625592417|
|                 1757| 8.993332283464566|
|                 1758|  8.13037054263566|
|                 1759|  9.26125771604938|
|                 1760|  8.53208176100629|
|                 1761| 8.940669242658425|
|      